In [ ]:
import pandas as pd
import numpy as np

In [2]:
basedf = pd.read_excel('dados_vendas.xlsx')

In [3]:
basedf.sort_values(['HORA'],ascending=[1]) #Ordenando os dados pela HORA
percentage=0.8
cutpoint=round(len(basedf)*percentage) #Dividindo a base em treinamento e teste
train,test=basedf[:cutpoint],basedf[cutpoint:]
print(len(train),len(test))

257393 64348


In [4]:
df= train[['COD_CLIENTE','COD_PRODUTO']] #Selecionando apenas as colunas que vão ser usadas
df=df.drop_duplicates(keep='first') #Retirando linhas duplicadas, clientes que compraram o mesmo produto mais de uma vez
v=df['COD_CLIENTE'].nunique() #Verificando o número de clientes
u=df['COD_PRODUTO'].nunique() #Verificando o número de produtos
convert_cliente={} #Array de conversão de ID do cliente para identificador inteiro começando por 0
revert_cliente=['' for x in range(v)]
c=0
for x in df['COD_CLIENTE'].unique():
    convert_cliente[x]=c
    revert_cliente[c]=x
    c+=1
convert_produto={}
revert_produto=['' for x in range(u)]
c=0
for x in df['COD_PRODUTO'].unique():
    convert_produto[x]=c
    revert_produto[c]=x
    c+=1
lista_adj=[[] for x in range(v)] #Inicialização da lista de adjacência
lista_adj_u=[[] for x in range(u)]
for i,j in df.iterrows():
    lista_adj[convert_cliente[j[0]]].append(convert_produto[j[1]])
    lista_adj_u[convert_produto[j[1]]].append(convert_cliente[j[0]])
outputname="Adjacency_list.txt"
text_file = open(outputname, "w")
text_file.write(str(v)+'\n')
text_file.write(str(u)+'\n')
for j in lista_adj:
    txt=str(j)[1:-1].replace(',',' ')
    text_file.write( txt+'\n')
text_file.close()


Nessa proxima seção está o código que chama o agrupador do professor gilberto, o código que está em C++ deve ser compilado previamente e a celula anterior que salva o arquivo Adjacency_list.txt também deve ter sido rodada ao menos uma vez.

In [5]:
import shlex
from subprocess import Popen, PIPE
def get_exitcode_stdout_stderr(cmd):
    """
    Execute the external command and get its exitcode, stdout and stderr.
    """
    args = shlex.split(cmd)

    proc = Popen(args, stdout=PIPE, stderr=PIPE)
    out, err = proc.communicate()
    exitcode = proc.returncode
    #
    return exitcode, out, err

cmd = './AMRPB Adjacency_list.txt'  # arbitrary external command, e.g. "python mytest.py"
exitcode, out, err = get_exitcode_stdout_stderr(cmd)

Na proxima celula é feita uma manipulação dos dados de saída do programa para eles serem melhor interpretados pelo recomendador, primeiro criamos o array da usando split no texto retornado e após isso são criadas as estruturas de bicluster 

In [6]:

f = out.decode("utf-8").split('\n')[-1]
nonnumbers=['(',')','}',']']
text=f.split('}')
clustertext=text[0].split('[')
solotext=text[1].split('(')
#print(len(solotext[2]))
biclusters=[]
v_cluster=[[] for x in range(v)]
u_cluster=[[] for x in range(u)]
#Clusters
for i,a in enumerate(clustertext[1:]):
    bicluster=[]
    for j,b in enumerate(a.split('),')):
        array=[]
        for k,c in enumerate(b.replace('],','').split(',')):
            txt=c
            for r in nonnumbers:
                txt=txt.replace(r, "")
            if(txt!=''):
                code=int(txt)
                array.append(code)
                if(not j):
                    v_cluster[code].append(i)
                else:
                    u_cluster[code].append(i)
        bicluster.append(array)
    biclusters.append(bicluster)
#Solos
#for i,a in enumerate(solotext[1:]):
#    for j,b in enumerate(a.split(',')):
#        txt=b
#        for r in nonnumbers:
#            txt=txt.replace(r, "")
#       if(txt!=''):
#           code=int(txt)
#            new_row = {'code':code, 'cluster':None }
#            if(j):
#               v_cluster[code].append()
#           else:
#                u_cluster[code].append()
print(u_cluster)

[[85, 90, 123], [67, 90, 106], [1, 66, 77, 79, 94, 95, 103, 105, 111, 117, 123], [], [], [87, 106], [88, 98], [], [], [98, 130], [2, 18, 61], [90, 108], [], [90], [112, 133], [], [12, 13, 14, 58, 75, 90, 94, 106, 108, 123, 131, 133], [], [], [1, 39, 46], [0, 15, 42, 55], [90], [], [], [67, 79], [], [86], [], [], [], [], [72, 108], [72, 98], [], [93], [], [], [], [], [99, 126], [125], [90], [], [], [], [], [], [67, 90, 109], [90], [], [98], [], [], [90], [17, 37, 57], [0, 10, 33, 61], [], [], [], [67, 103], [], [14, 18, 52], [67], [44, 49], [], [86], [107], [86, 88], [], [], [], [], [], [], [], [3, 36, 65], [], [17, 43, 57], [123], [], [13, 33, 54], [], [15, 18, 56], [86, 90], [], [67, 93, 116], [106], [], [], [], [16, 33, 46], [37, 53], [], [], [99], [6, 31, 59], [51], [8, 28, 49], [5, 24, 53], [], [11, 36, 48], [], [], [93], [], [99, 116], [], [], [], [8, 24, 63], [86], [], [14, 22, 51], [99], [43, 56], [14, 34, 60], [70], [88], [33], [], [], [], [], [], [], [], [], [36, 59], [16, 33,

Nessa parte foi feita uma função que determina a similaridade de dois produtos baseado na lista de adjacência u que é passada como parâmetro e os clusters gerados pelo agrupador

In [7]:
def similarity (cod_viewed,cod_recomended,lista_adj_u,cluster_u):
    total=len(cluster_u)
    partial=len(set(cluster_u) & set(lista_adj_u[cod_recomended]))
    #print(total,partial)
    #print(set(recomendations) & set(lista_adj_u[cod_recomended]),sep='\n')
    if(partial==0):
        return 0
    return partial/total
        

In [8]:
#def similarity (cod_viewed,cod_recomended,lista_adj_u,recomendations):
#    total=len(recomendations)
#    clusters
#    partial=len(set(recomendations) & set(lista_adj_u[cod_recomended]))
#    print(total,partial)
#    print(set(recomendations) & set(lista_adj_u[cod_recomended]),sep='\n')
#    if(partial==0):
#        return 0
#    return partial/total
        

A partir dessa seção temos os códigos que geram a recomendação para o sistema além de mensurações de performance

### Cliente

In [9]:
def recomenda_cliente(cliente,convert_cliente,v_cluster,biclusters):
    try:
        recomendations=[]
        cliente_key=convert_cliente[cliente]
        clusters=v_cluster[cliente_key]
        for a in clusters:
            for j,b in enumerate(biclusters[a]):
                if(j==1):
                    for x in b:
                        recomendations.append(x)
        return recomendations
    except KeyError:
        return 404 #"Cliente não encontrado"
    except:
        return -1 #"Erro inesperado encontrado, contacte o administrador"


### Produto

In [10]:
def recomenda_produto(produto,convert_produto,u_cluster,biclusters):
    try:
        recomendations=[]
        produto_key=convert_produto[produto]
        clusters=u_cluster[produto_key]
        scores=[]
        for a in clusters:
            for j,b in enumerate(biclusters[a]):
                if(j==1):
                    for x in b:
                        if(x not in recomendations and x!= produto_key):
                            recomendations.append(x)
                            scores.append(similarity(produto_key,x,lista_adj_u,b))
        return [recomendations,scores]
    except KeyError:
        return 404 #"Produto não encontrado"
    except:
        return -1 #"Erro inesperado encontrado, contacte o administrador"


### Usando vizinhos segundos para clientes

In [11]:
def recomenda_cliente_segundos(cliente,convert_cliente,v_cluster,biclusters):
    try:
        recomendations=[]
        cliente_key=convert_cliente[cliente]
        clusters=v_cluster[cliente_key]
        for a in clusters:
            for j,b in enumerate(biclusters[a]):
                if(j==1):
                    for x in b:
                        recomendations.append(x)
                else:
                    for viz in b:
                        for clu in v_cluster[viz]:
                            for i,el in enumerate(biclusters[clu]):
                                if(i==1):
                                    for pro in el:
                                        if(pro not in recomendations):
                                            #print(pro)
                                            recomendations.append(pro)
        #comprados=[]
        #compras_cliente=newdf[newdf.COD_CLIENTE==cliente]
        #for x in compras_cliente.COD_PRODUTO:
        #    comprados.append(convert_produto[x])
        #recomendations=list(filter(lambda a: a not in comprados, recomendations))
        return recomendations
    except KeyError:
        return 404 #"Cliente não encontrado"
    except:
        return -1 #"Erro inesperado encontrado, contacte o administrador"


### Usando vizinhos segundos para produtos

In [12]:
def recomenda_produto_segundos(produto):
    try:
        recomendations=[]
        produto_key=convert_produto[produto]
        clusters=u_cluster[cliente_key]
        for a in clusters:
            for j,b in enumerate(biclusters[a]):
                if(j==0):
                    for x in b:
                        recomendations.append(x)
                else:
                    for viz in b:
                        for clu in v_cluster[viz]:
                            for i,el in enumerate(biclusters[clu]):
                                if(i==1):
                                    for pro in el:
                                        if(pro not in recomendations):
                                            print(pro)
                                            recomendations.append(pro)
        print(recomendations)
    except KeyError:
        return 404 #"Produto não encontrado"
    except:
        return -1 #"Erro inesperado encontrado, contacte o administrador"


In [13]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
def convert_class(product,df):
    return df.loc[df['COD_PRODUTO'] == int(product)]['CLASSIFICACAO'].values[0]
    

# Teste de performance
#### Teste para recomendações partindo de produtos
<img src="DiagramaAMPB.jpg">

In [45]:
test= test[['COD_CLIENTE','COD_PRODUTO']]
test=test.drop_duplicates(keep='first')
v=test['COD_CLIENTE'].nunique()
u=test['COD_PRODUTO'].nunique()
print(test.shape)
test_client=test[test['COD_CLIENTE'].isin(convert_cliente)]
test_client.reset_index(drop=True,inplace=True)
test_client=test_client.astype('str')
test_client=test_client.groupby(by='COD_CLIENTE',as_index=False).agg(','.join)

test_product=test[test['COD_PRODUTO'].isin(convert_produto)]
test_product.reset_index(drop=True,inplace=True)
test_product=test_product.astype('str')
test_product=test_product.groupby(by='COD_PRODUTO',as_index=False).agg(','.join)

(62973, 2)


## Por Cliente

In [66]:
got_recomended=0
right_recomendations=0
buys_from_recomended=0
totalrec=0
correct_recomendation=0
for i,x in test_client.iterrows():
    total=len(x[1].split(','))
    compras=x[1].split(',')
    recomendations=recomenda_cliente_segundos(x[0],convert_cliente,v_cluster,biclusters)
    recomendations=[revert_produto[x] for x in recomendations]
    if(len(recomendations) >0):
        buys_from_recomended+=len(compras)
        got_recomended+=1
    recomended=np.intersect1d(compras,recomendations)
    totalrec+=len(recomendations)
    if(recomended.size >0):
        right_recomendations+=len(recomended)
        correct_recomendation+=1
print("Clientes na base de teste","\t\t|Clientes após filtros","\t\t|Clientes que receberam recomendações")
print(test['COD_CLIENTE'].nunique(),"\t\t\t\t\t|",len(test_client),"\t\t\t|",got_recomended)
print("Produtos Recomendados Corretamente \t|Total de Produtos recomendados\t|Porcentagem de acerto\t|Média de recomendações-cliente")
print(right_recomendations,"\t\t\t\t\t|",totalrec,'\t\t\t|',right_recomendations/totalrec*100,'\t|',totalrec/len(test_client))
print("Produtos comprados por quem recebeu recomendações\t|Porcentagem de clientes com alguma recomendação correta")
print(buys_from_recomended,"\t\t\t\t\t\t\t|",correct_recomendation/bought_and_got_recomended*100)

Clientes na base de teste 		|Clientes após filtros 		|Clientes que receberam recomendações
54681 					| 13152 			| 534
Produtos Recomendados Corretamente 	|Total de Produtos recomendados	|Porcentagem de acerto	|Média de recomendações-cliente
110 					| 84673 			| 0.12991154205000413 	| 6.438032238442823
Produtos comprados por quem recebeu recomendações	|Porcentagem de clientes com alguma recomendação correta
759 							| 12.51646903820817


#### Usando Classe

In [67]:
got_recomended=0
correct_recomendation=0
buys_from_recomended=0
right_recomendations=0
totalrec=0
for i,x in test_client.iterrows():
    total=len(x[1].split(','))
    compras=x[1].split(',')
    recomendations=recomenda_cliente_segundos(x[0],convert_cliente,v_cluster,biclusters)
    recomendations=[revert_produto[x] for x in recomendations]
    if(len(recomendations) >0):
        buys_from_recomended+=len(compras)
        got_recomended+=1
    else:
        continue
    recomendations=np.unique([convert_class(x,basedf) for x in recomendations])
    compras=[convert_class(x,basedf) for x in compras]
    recomended=np.intersect1d(compras,recomendations)
    totalrec+=len(recomendations)
    if(recomended.size >0):
        right_recomendations+=len(recomended)
        correct_recomendation+=1
print("Clientes na base de teste","\t\t|Clientes após filtros","\t\t|Clientes que receberam recomendações")
print(test['COD_CLIENTE'].nunique(),"\t\t\t\t\t|",len(test_client),"\t\t\t|",got_recomended)
print("Classes Recomendados Corretamente \t|Total de Produtos recomendados\t|Porcentagem de acerto\t|Média de recomendações-cliente")
print(right_recomendations,"\t\t\t\t\t|",totalrec,'\t\t\t|',right_recomendations/totalrec*100,'\t|',totalrec/len(test_client))
print("Produtos comprados por quem recebeu recomendações\t|Porcentagem de clientes com alguma recomendação correta")
print(buys_from_recomended,"\t\t\t\t\t\t\t|",correct_recomendation/buys_from_recomended*100)

Clientes na base de teste 		|Clientes após filtros 		|Clientes que receberam recomendações
54681 					| 13152 			| 534
Classes Recomendados Corretamente 	|Total de Produtos recomendados	|Porcentagem de acerto	|Média de recomendações-cliente
623 					| 11813 			| 5.27385084229239 	| 0.8981903892944039
Produtos comprados por quem recebeu recomendações	|Porcentagem de clientes com alguma recomendação correta
759 							| 67.3254281949934


## Quem comprou também comprou

In [54]:

right_recomendations=0
correct_recomendation=0
totalrec=0
cliente_search=test.reset_index(drop=True)
cliente_search=cliente_search.astype('str')
cliente_search=cliente_search.groupby(by='COD_CLIENTE',as_index=False).agg(','.join)
cliente_search=cliente_search.set_index('COD_CLIENTE')
for i,x in test_product.iterrows():
    compras=[]
    total=len(x[1].split(','))
    compradores=x[1].split(',')
    for y in compradores:
        for z in cliente_search.loc[y].values:
            for w in z.split(','):
                if(w not in compras and w!=x[0]):
                    compras.append(w)
    recomendations=recomenda_produto(int(x[0]),convert_produto,u_cluster,biclusters)[0]
    recomendations=[revert_produto[x] for x in recomendations]
    recomended=np.intersect1d(compras,recomendations)
    totalrec+=len(recomendations)
    if(recomended.size >0):
        right_recomendations+=len(recomended)
        correct_recomendation+=1
    #print(x[0],compras,recomendations)
print("Produtos na base de teste","\t|Produtos após filtros","\t|Produtos Recomendados Corretamente")
print(test['COD_PRODUTO'].nunique(),"\t\t\t\t|",len(test_product),"\t\t\t|",right_recomendations)
print("Total de Produtos recomendados\t|Porcentagem de acerto\t|Média de recomendações-produto")
print(totalrec,'\t\t\t|',right_recomendations/totalrec*100,'\t\t\t|',totalrec/len(test_client))
print("Porcentagem de produtos com alguma recomendação correta")
print(correct_recomendation/len(test_product)*100)

Produtos na base de teste 	|Produtos após filtros 	|Produtos Recomendados Corretamente
459 				| 378 			| 278
Total de Produtos recomendados	|Porcentagem de acerto	|Média de recomendações-produto
2173 			| 12.793373216751036 			| 0.1652220194647202
Porcentagem de produtos com alguma recomendação correta
21.164021164021165


#### Usando Classes

In [55]:
right_recomendations=0
totalrec=0
correct_recomendation=0
cliente_search=test.reset_index(drop=True)
cliente_search=cliente_search.astype('str')
cliente_search=cliente_search.groupby(by='COD_CLIENTE',as_index=False).agg(','.join)
cliente_search=cliente_search.set_index('COD_CLIENTE')
for i,x in test_product.iterrows():
    compras=[]
    total=len(x[1].split(','))
    compradores=x[1].split(',')
    for y in compradores:
        for z in cliente_search.loc[y].values:
            for w in z.split(','):
                if(w not in compras and w!=x[0]):
                    compras.append(w)
    recomendations=recomenda_produto(int(x[0]),convert_produto,u_cluster,biclusters)[0]
    recomendations=[revert_produto[x] for x in recomendations]
    #recomendations=basedf.loc[basedf['COD_PRODUTO'].isin(recomendations)]['CLASSIFICACAO'].unique()
    #compras=basedf.loc[basedf['COD_PRODUTO'].isin(compras)]['CLASSIFICACAO'].unique()
    recomendations=np.unique([convert_class(x,basedf) for x in recomendations])
    totalrec+=len(recomendations)
    compras=[convert_class(x,basedf) for x in compras]
    #print(len(compradores))
    recomended=np.intersect1d(compras,recomendations)
    if(recomended.size>0):
        right_recomendations+=len(recomended)
        correct_recomendation+=1
    #print(x[0],compras,recomendations)
print("Produtos na base de teste","\t|Produtos após filtros","\t|Classes Recomendados Corretamente")
print(test['COD_PRODUTO'].nunique(),"\t\t\t\t|",len(test_product),"\t\t\t|",right_recomendations)
print("Total de Produtos recomendados\t|Porcentagem de acerto\t|Média de recomendações-produto")
print(totalrec,'\t\t\t|',right_recomendations/totalrec*100,'\t\t\t|',totalrec/len(test_client))
print("Porcentagem de produtos com alguma recomendação correta")
print(correct_recomendation/len(test_product)*100)

Produtos na base de teste 	|Produtos após filtros 	|Classes Recomendados Corretamente
459 				| 378 			| 460
Total de Produtos recomendados	|Porcentagem de acerto	|Média de recomendações-produto
1135 			| 40.52863436123348 			| 0.08629866180048662
Porcentagem de produtos com alguma recomendação correta
45.23809523809524
